In [1]:
# !pip install transformers accelerate
# from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import Dataset
# %pip install evaluate
import evaluate
from sklearn.model_selection import train_test_split
# from ipywidgets import FloatProgress
import csv
from optuna import Trial
from typing import Dict, Union, Any
import os
import sys

# notebook_login()

csv.field_size_limit(500 * 1024 * 1024)
CUDA_LAUNCH_BLOCKING=1
# get pwd
notebook_path = os.path.abspath('')

# Find the part of the path that contains 'commitFit'
commit_fit_path = None
for part in notebook_path.split(os.sep):
    print(part)
    if 'CommitFit' in part:
        commit_fit_path = notebook_path.split(part)[0] + part
        break

if commit_fit_path is None:
    raise ValueError("Path containing 'commitFit' not found in notebook path.")

# Add commitFit directory to Python path, so we can import moudule from commitfit folder directly
if commit_fit_path not in sys.path:
    sys.path.append(commit_fit_path)

from commitfit import CommitFitModel, CommitFitTrainer 
from commitfit import get_templated_dataset,sample_dataset


root
autodl-tmp
CommitFit


In [2]:
df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/multi-lang-exp/dataset.csv', index_col=0, encoding='utf_8_sig')
# df.fillna('', inplace=True)
label2id={'C':0, 'A':1, 'P':2}
# df.reset_index(inplace=True)
df = df.rename(columns={'maintenance_type':'label','msgs':'text','commit_diff':'diffs'})
df = df.replace({"label": label2id})
# df = df['diffs']
# # print(df)

df
# test_sample = df.sample(n=3, random_state=1)

/tmp/ipykernel_2758/947145482.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": label2id})


,language,commit,commit_url,label,user,repo,feature,diffs,text
0,C++,be937a3290223f926fe50684f1344569a573ed4b,https://github.com/tensorflow/tensorflow/commi...,0,tensorflow,tensorflow,"[1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 6, ...",diff --git a/tensorflow/compiler/mlir/lite/fla...,Fixing per axis quantization bug in flatbuffer...
1,C++,c800d2e36954edddcb83aa1df7f623f2780c7780,https://github.com/tensorflow/tensorflow/commi...,1,tensorflow,tensorflow,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/tensorflow/python/tools/saved_mod...,Add support for python expression as SavedMode...
2,C++,469e56eeff17eb857c95e935ca7b49723c43470e,https://github.com/tensorflow/tensorflow/commi...,2,tensorflow,tensorflow,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/.github/ISSUE_TEMPLATE/00-bug-iss...,Fix GitHub issue templates. Actual fix for #36...
3,C++,24f5d302c0906f3377f4d6bcc2735d943ce13e3d,https://github.com/tensorflow/tensorflow/commi...,1,tensorflow,tensorflow,"[3, 37, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/tensorflow/compiler/xla/pjrt/dist...,"Add new PjRT distributed APIs (KeyValueDelete,..."
4,C++,1effdb7b959503596b07f7f3e74618ab63f9e5e5,https://github.com/tensorflow/tensorflow/commi...,2,tensorflow,tensorflow,"[1, 0, 12, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/tensorflow/core/common_runtime/ea...,Remove IsLocalDevice() since it's no longer us...
...,...,...,...,...,...,...,...,...,...
1576,Java,e4bbfd537a63a52a6094491504fc7b0bb54bcb0e,NaN,0,kiegroup,drools,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/drools-persistence-jpa/src/main/j...,fixing reflection constructor to use int inste...
1577,Java,75b81dcbd26a11d5b370ea5bdf102c2d62ebd0c3,NaN,0,kiegroup,drools,"[2, 6, 2, 3, 0, 0, 0, 1, 7, 0, 0, 0, 0, 0, 0, ...",diff --git a/drools-compiler/src/main/java/org...,Fix OutOfBoundException on- MemoryFileSystem.e...
1578,Java,6bc9677bbe919cd5bcadd8af2f5b0c838757a4ce,NaN,0,kiegroup,drools,"[1, 1, 5, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/drools-core/src/main/java/org/dro...,[DROOLS-293] fix ObjectTypeNode id creation an...
1579,Java,48701edad8513b27acec7216581e64637157c86a,NaN,1,kiegroup,drools,"[2, 24, 2, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/drools-core/src/main/java/org/dro...,DROOLS-515 Kie-Camel is broken after Camel Upd...


In [3]:
train, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test = train_test_split(temp_df, test_size=0.5, random_state=42)

In [4]:
train = train.groupby("label", group_keys=False).apply(lambda x: x.sample(n=20, random_state=42))

/tmp/ipykernel_2758/240346103.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train = train.groupby("label", group_keys=False).apply(lambda x: x.sample(n=20, random_state=42))


In [5]:
train

,language,commit,commit_url,label,user,repo,feature,diffs,text
781,Python,a2479f46f3d05b37254ad701b6b76f84624d3cb5,https://github.com/django/django/commit/a2479f...,0,django,django,"[3, 13, 0, 3, 1, 0, 0, 1, 5, 0, 0, 0, 0, 0, 0,...",diff --git a/django/contrib/auth/migrations/00...,Fixed #7220 -- Allowed AbstractBaseUser.last_l...
126,C++,6662fe7b185aab29aa4b2c5e08784e9da75bc700,https://github.com/ocornut/imgui/commit/6662fe...,0,ocornut,imgui,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/stb_truetype.h b/stb_truetype.h\n...,stb_truetype: Fixed unused variable warnings f...
1323,TypeScript,177f1fc5dd489e90eb1c4cf055e342fbc4f6ef32,https://github.com/apache/echarts/commit/177f1...,0,apache,echarts,"[1, 0, 0, 1, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/src/component/toolbox/feature/Sav...,fix(toolbox): remove outer link and fix issues...
683,JavaScript,ec7be48470e42472a07572b7f1ca2abc91bfdad3,https://github.com/chartjs/Chart.js/commit/ec7...,0,chartjs,Chart.js,"[1, 6, 0, 0, 2, 0, 0, 2, 34, 0, 0, 0, 0, 0, 0,...",diff --git a/src/controllers/controller.scatte...,Fix toggling showLine option on scatter contro...
871,Python,daf4c358f7d382e7fc0466b06e3c528d24b3f760,https://github.com/ansible/ansible/commit/daf4...,0,ansible,ansible,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/library/packaging/pip b/library/p...,fixes case where name is omitted from pip arg ...
681,JavaScript,8f6f8820686cc57ab0ee3139c4d7d7c2c13b7014,https://github.com/chartjs/Chart.js/commit/8f6...,0,chartjs,Chart.js,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/src/elements/element.point.js b/s...,Fix jshint issue\n\n
1402,Java,ca2f1678d75d9d42867f2124fa3e7dfcf1c367f7,NaN,0,apache,hbase,"[0, 0, 0, 0, 0, 0, 0, 1, 4, 0, 0, 2, 0, 0, 0, ...",diff --git a/CHANGES.txt b/CHANGES.txt\nindex ...,HBASE-2757. Fix flaky TestFromClientSide test ...
1072,Rust,33a4df8008ce138106a71b047ddef3ba57a0a28b,https://github.com/bevyengine/bevy/commit/33a4...,0,bevyengine,bevy,"[2, 9, 12, 15, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0...",diff --git a/crates/bevy_text/src/lib.rs b/cra...,Update layout/style when scale factor changes ...
552,JavaScript,0c7236b292a77e330710994417c4214475fb6880,https://github.com/axios/axios/commit/0c7236b2...,0,axios,axios,"[1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/lib/axios.js b/lib/axios.js\ninde...,Fixing issue referencing wrong headers\n\n
395,Go,7b48138ad0afc11d1ebbaa86095127e8384d6d42,https://github.com/Dreamacro/clash/commit/7b48...,0,Dreamacro,clash,"[9, 43, 14, 26, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/adapters/outbound/base.go b/adapt...,Fix: vmess udp crash\n\n


In [6]:
# train.to_csv('/root/autodl-tmp/COLARE-main/dataset/multi-lang-exp/20_train.csv',index=False)
# test.to_csv('/root/autodl-tmp/COLARE-main/dataset/multi-lang-exp/test.csv',index=False)
# val.to_csv('/root/autodl-tmp/COLARE-main/dataset/multi-lang-exp/eval.csv',index=False)
# train.to_csv('/root/autodl-tmp/CommitFit/dataset/multi-lang-exp/20_train.csv',index=False)
# test.to_csv('/root/autodl-tmp/CommitFit/dataset/multi-lang-exp/test.csv',index=False)
# val.to_csv('/root/autodl-tmp/CommitFit/dataset/multi-lang-exp/eval.csv',index=False)

In [7]:
Dataset_train = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [8]:
train_dataset = get_templated_dataset(Dataset_train, candidate_labels=['Corrective','Adaptive','Perfective'], sample_size=8)

In [9]:
train_code_change = list(train['diffs'].astype(str))
test_code_change = list(test['diffs'].astype(str))

In [10]:
# list(train['message'].astype(str).values)
train_dataset

Dataset({
    features: ['language', 'commit', 'commit_url', 'label', 'user', 'repo', 'feature', 'diffs', 'text', '__index_level_0__'],
    num_rows: 84
})

In [11]:
len(train)

60

In [12]:
train['label'].value_counts()

label
0    20
1    20
2    20
Name: count, dtype: int64

In [13]:
test['label'].value_counts()

label
2    155
0     51
1     32
Name: count, dtype: int64

In [14]:
len(train_dataset)

84

In [15]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    precision_score = metrics.precision_score(y_test,y_pred, average='macro')
    recall_score = metrics.recall_score(y_test,y_pred,average='macro')
    f1_score = metrics.f1_score(y_test,y_pred,average='macro')
    
    # 混淆矩阵
    cm = metrics.confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:\n", cm)
    print(metrics.classification_report(y_test, y_pred, digits=4))
    # return {"classification_report": classification_report}
    return {"classification_report":metrics.classification_report(y_test, y_pred, digits=4),"Confusion Matrix":cm,"precision": round(precision_score, 4),"recall": round(recall_score, 4), "f1":round(f1_score, 4), "accuracy": round(accuracy_score, 4)}

In [16]:
train_dataset

Dataset({
    features: ['language', 'commit', 'commit_url', 'label', 'user', 'repo', 'feature', 'diffs', 'text', '__index_level_0__'],
    num_rows: 84
})

In [17]:
train

,language,commit,commit_url,label,user,repo,feature,diffs,text
781,Python,a2479f46f3d05b37254ad701b6b76f84624d3cb5,https://github.com/django/django/commit/a2479f...,0,django,django,"[3, 13, 0, 3, 1, 0, 0, 1, 5, 0, 0, 0, 0, 0, 0,...",diff --git a/django/contrib/auth/migrations/00...,Fixed #7220 -- Allowed AbstractBaseUser.last_l...
126,C++,6662fe7b185aab29aa4b2c5e08784e9da75bc700,https://github.com/ocornut/imgui/commit/6662fe...,0,ocornut,imgui,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/stb_truetype.h b/stb_truetype.h\n...,stb_truetype: Fixed unused variable warnings f...
1323,TypeScript,177f1fc5dd489e90eb1c4cf055e342fbc4f6ef32,https://github.com/apache/echarts/commit/177f1...,0,apache,echarts,"[1, 0, 0, 1, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/src/component/toolbox/feature/Sav...,fix(toolbox): remove outer link and fix issues...
683,JavaScript,ec7be48470e42472a07572b7f1ca2abc91bfdad3,https://github.com/chartjs/Chart.js/commit/ec7...,0,chartjs,Chart.js,"[1, 6, 0, 0, 2, 0, 0, 2, 34, 0, 0, 0, 0, 0, 0,...",diff --git a/src/controllers/controller.scatte...,Fix toggling showLine option on scatter contro...
871,Python,daf4c358f7d382e7fc0466b06e3c528d24b3f760,https://github.com/ansible/ansible/commit/daf4...,0,ansible,ansible,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/library/packaging/pip b/library/p...,fixes case where name is omitted from pip arg ...
681,JavaScript,8f6f8820686cc57ab0ee3139c4d7d7c2c13b7014,https://github.com/chartjs/Chart.js/commit/8f6...,0,chartjs,Chart.js,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/src/elements/element.point.js b/s...,Fix jshint issue\n\n
1402,Java,ca2f1678d75d9d42867f2124fa3e7dfcf1c367f7,NaN,0,apache,hbase,"[0, 0, 0, 0, 0, 0, 0, 1, 4, 0, 0, 2, 0, 0, 0, ...",diff --git a/CHANGES.txt b/CHANGES.txt\nindex ...,HBASE-2757. Fix flaky TestFromClientSide test ...
1072,Rust,33a4df8008ce138106a71b047ddef3ba57a0a28b,https://github.com/bevyengine/bevy/commit/33a4...,0,bevyengine,bevy,"[2, 9, 12, 15, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0...",diff --git a/crates/bevy_text/src/lib.rs b/cra...,Update layout/style when scale factor changes ...
552,JavaScript,0c7236b292a77e330710994417c4214475fb6880,https://github.com/axios/axios/commit/0c7236b2...,0,axios,axios,"[1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/lib/axios.js b/lib/axios.js\ninde...,Fixing issue referencing wrong headers\n\n
395,Go,7b48138ad0afc11d1ebbaa86095127e8384d6d42,https://github.com/Dreamacro/clash/commit/7b48...,0,Dreamacro,clash,"[9, 43, 14, 26, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/adapters/outbound/base.go b/adapt...,Fix: vmess udp crash\n\n


In [18]:
model_id = "../../sentence-transformers/paraphrase-mpnet-base-v2"

In [19]:
def hp_space(trial: Trial) -> Dict[str, Union[float, int, str]]:
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-3, log=True),
        # "num_epochs": trial.suggest_int("num_epochs", 1, 3),
        # "batch_size": trial.suggest_categorical("batch_size", [8, 12, 16]),
        # "seed": trial.suggest_int("seed", 1, 40),
        # "num_iterations": trial.suggest_int("num_iterations", 10, 20),
        # "solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear"]),
    }
def model_init(params: Dict[str, Any]) -> CommitFitModel:
    params = params or {}
    # learning_rate = params.get("learning_rate")
    # num_iterations = params.get("num_iterations", 20)
    # solver = params.get("solver", "liblinear")
    # params = {
    #     "head_params": {
    #         # "max_iter": num_iterations,
    #         # "solver": solver,
    #     }
    # }
    return CommitFitModel.from_pretrained(model_id, **params)
def my_compute_objective(metrics):
    print('+++++++++++',metrics)
    return  metrics['accuracy']

In [20]:
trainer = CommitFitTrainer(
    train_dataset=train_dataset,
    train_code_change = train_code_change,
    test_code_change = test_code_change,
    eval_dataset=test_dataset,
    model_init=model_init,
    metric = compute_metrics,
    num_iterations=20,
    num_epochs=1
)
best_run = trainer.hyperparameter_search(direction="maximize", hp_space=hp_space, compute_objective=my_compute_objective, n_trials=5)

model_head.pkl not found in /root/autodl-tmp/CommitFit/sentence-transformers/paraphrase-mpnet-base-v2, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
[I 2025-08-29 19:13:51,590] A new study created in memory with name: no-name-93ca4576-a2cb-46bf-9e30-292ee41e2da3
Trial: {'learning_rate': 1.6851708656253302e-06}
model_head.pkl not found in /root/autodl-tmp/CommitFit/sentence-transformers/paraphrase-mpnet-base-v2, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 3360
  Num epochs = 1
  Total optimization steps = 210
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/210 [00:00<?, ?it/s]

...

***** Running training *****
  Num examples = 3360
  Num epochs = 1
  Total optimization steps = 210
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/210 [00:00<?, ?it/s]

***** Running evaluation *****
/root/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/root/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/root/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Confusion Matrix:
 [[ 51   0   0]
 [ 32   0   0]
 [155   0   0]]
              precision    recall  f1-score   support

           0     0.2143    1.0000    0.3529        51
           1     0.0000    0.0000    0.0000        32
           2     0.0000    0.0000    0.0000       155

    accuracy                         0.2143       238
   macro avg     0.0714    0.3333    0.1176       238
weighted avg     0.0459    0.2143    0.0756       238

+++++++++++ {'classification_report': '              precision    recall  f1-score   support\n\n           0     0.2143    1.0000    0.3529        51\n           1     0.0000    0.0000    0.0000        32\n           2     0.0000    0.0000    0.0000       155\n\n    accuracy                         0.2143       238\n   macro avg     0.0714    0.3333    0.1176       238\nweighted avg     0.0459    0.2143    0.0756       238\n', 'Confusion Matrix': array([[ 51,   0,   0],
       [ 32,   0,   0],
       [155,   0,   0]]), 'precision': 0.0714, 'recall'

In [21]:
best_run

BestRun(run_id='1', objective=0.6849, hyperparameters={'learning_rate': 3.058569187188438e-05}, backend=<optuna.study.study.Study object at 0x7fafd70e1330>)

In [22]:
best_run.hyperparameters

{'learning_rate': 3.058569187188438e-05}

In [23]:
trainer.apply_hyperparameters(best_run.hyperparameters, final_model=True)
trainer.train()

model_head.pkl not found in /root/autodl-tmp/CommitFit/sentence-transformers/paraphrase-mpnet-base-v2, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 3360
  Num epochs = 1
  Total optimization steps = 210
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/210 [00:00<?, ?it/s]

In [24]:
fewshot_metrics = trainer.evaluate()
fewshot_metrics

***** Running evaluation *****


Confusion Matrix:
 [[40  2  9]
 [ 1 26  5]
 [28 30 97]]
              precision    recall  f1-score   support

           0     0.5797    0.7843    0.6667        51
           1     0.4483    0.8125    0.5778        32
           2     0.8739    0.6258    0.7293       155

    accuracy                         0.6849       238
   macro avg     0.6340    0.7409    0.6579       238
weighted avg     0.7536    0.6849    0.6955       238



{'classification_report': '              precision    recall  f1-score   support\n\n           0     0.5797    0.7843    0.6667        51\n           1     0.4483    0.8125    0.5778        32\n           2     0.8739    0.6258    0.7293       155\n\n    accuracy                         0.6849       238\n   macro avg     0.6340    0.7409    0.6579       238\nweighted avg     0.7536    0.6849    0.6955       238\n',
 'Confusion Matrix': array([[40,  2,  9],
        [ 1, 26,  5],
        [28, 30, 97]]),
 'precision': 0.634,
 'recall': 0.7409,
 'f1': 0.6579,
 'accuracy': 0.6849}

In [25]:
#发送多种类型的邮件
from email.mime.multipart import MIMEMultipart
import smtplib

from email.mime.text import MIMEText
msg_from = '915803745@qq.com'  # 发送方邮箱
passwd = 'vcuosuurrgkfbdai'   #就是上面的授权码
 
# to= ['g.zhang@gotion.com', 'j.tong@gotion.com'] #接受方邮箱
to= ['j.tong@gotion.com'] #接受方邮箱
#设置邮件内容
#MIMEMultipart类可以放任何内容
msg = MIMEMultipart()
conntent=f"{fewshot_metrics}"
#把内容加进去
msg.attach(MIMEText(conntent,'plain','utf-8'))
 
#设置邮件主题
msg['Subject']="强化学习模型训练完毕"
 
#发送方信息
msg['From']=msg_from
 
#开始发送
 
#通过SSL方式发送，服务器地址和端口
s = smtplib.SMTP_SSL("smtp.qq.com", 465)
# 登录邮箱
s.login(msg_from, passwd)
#开始发送
s.sendmail(msg_from,to,msg.as_string())
print("强化学习模型训练完毕")

强化学习模型训练完毕
